In [1]:
import os, sys, time, requests, zipfile, io, re
import numpy as np, pandas as pd, geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import folium
import osmnx as ox

# spatial stats & ML
import libpysal, esda
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import shap
import joblib
from bs4 import BeautifulSoup

WORK_DIR = "/content/stats19_mumbai_demo"  # change as needed
os.makedirs(WORK_DIR, exist_ok=True)

print("Working directory:", WORK_DIR)
print("Data source: UK Department for Transport - Road Safety Data (STATS19). See gov.uk Road Safety Data page.")


C:\Users\Croma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Working directory: /content/stats19_mumbai_demo
Data source: UK Department for Transport - Road Safety Data (STATS19). See gov.uk Road Safety Data page.


In [2]:
gov_url = "https://www.gov.uk/government/statistics/road-safety-data"
print("Fetching GOV.UK Road Safety Data page:", gov_url)
r = requests.get(gov_url, timeout=30)
r.raise_for_status()
soup = BeautifulSoup(r.text, "html.parser")

# find anchor links whose text contains 'Road Safety Data - Collisions' or link href ends with .csv or .zip
links = []
for a in soup.find_all("a", href=True):
    href = a['href']
    text = a.get_text(separator=" ").strip()
    if (("Collisions" in text or "collisions" in text) and href.lower().endswith(".csv")) or href.lower().endswith(".zip"):
        links.append((text, href))

# If none found directly, look for CSV filenames present in page text lines (some pages embed direct resource blocks)
if not links:
    for a in soup.find_all("a", href=True):
        href = a['href']
        # absolute
        if href.lower().endswith(".csv") or href.lower().endswith(".zip"):
            links.append((a.get_text(), href))

print("Found candidate files (sample):")
for t,h in links[:10]:
    print("-", t, h)

# Make hrefs absolute if needed
base = "https://www.gov.uk"
links = [(t, (h if h.startswith("http") else base + h)) for t,h in links]
# Prioritize a 'Collisions' CSV for latest year (search text)
chosen = None
for t,h in links:
    if 'Collisions' in t or 'collisions' in t:
        chosen = (t,h)
        break
if not chosen and links:
    chosen = links[0]

if not chosen:
    raise RuntimeError("No collisions CSV/zip found on GOV.UK page automatically. Please download manually and place in WORK_DIR.")

print("Chosen file to download:", chosen)


Fetching GOV.UK Road Safety Data page: https://www.gov.uk/government/statistics/road-safety-data
Found candidate files (sample):
- Road Safety Data - Collisions - 2024 https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-collision-2024.csv
- Road Safety Data - Collisions - 1979 - Latest Published Year https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-collision-1979-latest-published-year.csv
- Road Safety Data - Collisions - last 5 years https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-collision-last-5-years.csv
- Road Safety Data - Collisions - 2023 https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-collision-2023.csv
- Road Safety Data - Collisions - 2022 https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-collision-2022.csv
- Road Safety Data - Collisions - 2021 https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-

In [3]:
text, url = chosen
print("Downloading:", url)
resp = requests.get(url, stream=True, timeout=60)
resp.raise_for_status()

fname = os.path.join(WORK_DIR, url.split("/")[-1].split("?")[0])
print("Saving to:", fname)
with open(fname, "wb") as fh:
    for chunk in resp.iter_content(chunk_size=8192):
        if chunk:
            fh.write(chunk)

# If it's a zip, inspect and extract CSV inside
if fname.lower().endswith(".zip"):
    print("Zip downloaded; listing contents...")
    with zipfile.ZipFile(fname, 'r') as z:
        z.printdir()
        # pick entry with 'Collisions' or large collisions CSV if present
        candidates = [n for n in z.namelist() if 'Collisions' in n or 'collisions' in n or n.lower().endswith('.csv')]
        if not candidates:
            candidates = z.namelist()
        # choose first CSV
        csvname = candidates[0]
        print("Extracting:", csvname)
        z.extract(csvname, path=WORK_DIR)
        csv_path = os.path.join(WORK_DIR, csvname)
else:
    # assume csv
    csv_path = fname

print("CSV path:", csv_path)


Downloading: https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-collision-2024.csv
Saving to: /content/stats19_mumbai_demo\dft-road-casualty-statistics-collision-2024.csv
CSV path: /content/stats19_mumbai_demo\dft-road-casualty-statistics-collision-2024.csv


In [4]:
# read a small sample to inspect columns (safe for large files)
sample = pd.read_csv(csv_path, nrows=200)
print("Preview columns:", sample.columns.tolist()[:60])
sample.head()


Preview columns: ['collision_index', 'collision_year', 'collision_ref_no', 'location_easting_osgr', 'location_northing_osgr', 'longitude', 'latitude', 'police_force', 'collision_severity', 'number_of_vehicles', 'number_of_casualties', 'date', 'day_of_week', 'time', 'local_authority_district', 'local_authority_ons_district', 'local_authority_highway', 'local_authority_highway_current', 'first_road_class', 'first_road_number', 'road_type', 'speed_limit', 'junction_detail_historic', 'junction_detail', 'junction_control', 'second_road_class', 'second_road_number', 'pedestrian_crossing_human_control_historic', 'pedestrian_crossing_physical_facilities_historic', 'pedestrian_crossing', 'light_conditions', 'weather_conditions', 'road_surface_conditions', 'special_conditions_at_site', 'carriageway_hazards_historic', 'carriageway_hazards', 'urban_or_rural_area', 'did_police_officer_attend_scene_of_accident', 'trunk_road_flag', 'lsoa_of_accident_location', 'enhanced_severity_collision', 'collisio

,collision_index,collision_year,collision_ref_no,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,collision_severity,number_of_vehicles,...,carriageway_hazards_historic,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,enhanced_severity_collision,collision_injury_based,collision_adjusted_severity_serious,collision_adjusted_severity_slight
0,202417H103224,2024,17H103224,448894,532505,-1.24312,54.68523,17,3,2,...,-1,0,1,2,2,E01011983,3,1,0.000000,1.000000
1,202417M217924,2024,17M217924,452135,519436,-1.19517,54.56747,17,2,2,...,-1,0,1,3,2,E01012061,7,1,1.000000,0.000000
2,202417S204524,2024,17S204524,445427,522924,-1.29837,54.59946,17,3,2,...,0,0,2,1,2,E01012280,-1,0,0.111621,0.888379
3,2024481510889,2024,481510889,533587,181174,-0.07626,51.51371,48,2,1,...,-1,0,1,1,2,E01000005,7,1,1.000000,0.000000
4,2024481563500,2024,481563500,532676,180902,-0.08948,51.51148,48,2,1,...,-1,0,1,1,2,E01032739,5,1,1.000000,0.000000


In [5]:
# load fully (file is ~20MB for a year; large full historic file ~1.4GB)
print("Loading full collisions CSV (this may take a moment)...")
df = pd.read_csv(csv_path, low_memory=False)
print("Rows:", len(df))
# common STATS19 coordinate columns: 'Longitude', 'Latitude' (decimal degrees)
coord_cols = [c for c in df.columns if c.lower() in ('longitude','latitude','long','lat','eastings','northings')]
print("Detected coord-related columns:", coord_cols)

# prefer decimal lon/lat
if 'Longitude' in df.columns and 'Latitude' in df.columns:
    df = df.dropna(subset=['Longitude','Latitude'])
    df['longitude'] = df['Longitude']
    df['latitude'] = df['Latitude']
else:
    # try lower-case variants
    for c in ['longitude','lat','long','latitude']:
        if c in df.columns:
            if c in ('long','longitude'): df['longitude'] = df[c]
            if c in ('lat','latitude'): df['latitude'] = df[c]
    # If no lon/lat, check for Eastings/Northings and convert (STATS19 sometimes includes easting/northing)
    if ('Easting' in df.columns and 'Northing' in df.columns) or ('easting' in df.columns and 'northing' in df.columns):
        e_col = 'Easting' if 'Easting' in df.columns else 'easting'
        n_col = 'Northing' if 'Northing' in df.columns else 'northing'
        # convert OSGB Easting/Northing to lat/lon using pyproj
        from pyproj import Transformer
        transformer = Transformer.from_crs("epsg:27700", "epsg:4326", always_xy=True)
        lons,lats = transformer.transform(df[e_col].values, df[n_col].values)
        df['longitude'] = lons
        df['latitude'] = lats

# drop any rows missing lat/lon
df = df.dropna(subset=['longitude','latitude'])
print("Rows with coordinates:", len(df))


Loading full collisions CSV (this may take a moment)...
Rows: 100927
Detected coord-related columns: ['longitude', 'latitude']
Rows with coordinates: 100927


In [6]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']), crs='epsg:4326')
gdf = gdf.to_crs(epsg=3857)   # metric CRS for grid and lengths
print("GeoDataFrame rows:", len(gdf))
gdf.head()


GeoDataFrame rows: 100927


,collision_index,collision_year,collision_ref_no,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,collision_severity,number_of_vehicles,...,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,enhanced_severity_collision,collision_injury_based,collision_adjusted_severity_serious,collision_adjusted_severity_slight,geometry
0,202417H103224,2024,17H103224,448894,532505,-1.24312,54.68523,17,3,2,...,0,1,2,2,E01011983,3,1,0.000000,1.000000,POINT (-138383.485 7301013.779)
1,202417M217924,2024,17M217924,452135,519436,-1.19517,54.56747,17,2,2,...,0,1,3,2,E01012061,7,1,1.000000,0.000000,POINT (-133045.716 7278369.361)
2,202417S204524,2024,17S204524,445427,522924,-1.29837,54.59946,17,3,2,...,0,2,1,2,E01012280,-1,0,0.111621,0.888379,POINT (-144533.887 7284514.331)
3,2024481510889,2024,481510889,533587,181174,-0.07626,51.51371,48,2,1,...,0,1,1,2,E01000005,7,1,1.000000,0.000000,POINT (-8489.224 6712671.106)
4,2024481563500,2024,481563500,532676,180902,-0.08948,51.51148,48,2,1,...,0,1,1,2,E01032739,5,1,1.000000,0.000000,POINT (-9960.868 6712272.222)


In [8]:
df_sample = df[['longitude','latitude']].copy()
print(df_sample.describe())
print("Min/Max lon/lat:", df_sample.longitude.min(), df_sample.longitude.max(), df_sample.latitude.min(), df_sample.latitude.max())


           longitude       latitude
count  100927.000000  100927.000000
mean       -1.229156      52.383155
std         1.356707       1.319833
min        -7.388680      49.912210
25%        -2.124290      51.464760
50%        -1.141100      51.889930
75%        -0.139255      53.347150
max         1.756220      60.344800
Min/Max lon/lat: -7.38868 1.75622 49.91221 60.3448


In [10]:
# ========== SAFER GRID BUILD (REPLACE Cell 7) ==========
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon

# df is your loaded STATS19 pandas DataFrame with 'longitude' & 'latitude'
# If df not defined, load a small sample first or re-run the CSV load cell.

# 1) Filter to valid UK-like coords (defensive)
df_valid = df[(df['longitude'] > -10) & (df['longitude'] < 5) &
              (df['latitude'] > 49) & (df['latitude'] < 61)].copy()
print("Rows before filter:", len(df), "after filter:", len(df_valid))
if len(df_valid) == 0:
    raise RuntimeError("No valid UK coordinates found after filtering. Check your longitude/latitude columns.")

# 2) Convert to GeoDataFrame in metric CRS (EPSG:3857)
gdf = gpd.GeoDataFrame(df_valid, geometry=gpd.points_from_xy(df_valid.longitude, df_valid.latitude), crs="epsg:4326")
gdf = gdf.to_crs(epsg=3857)

# 3) Build a compact area to grid: convex hull + buffer (meters)
hull = gdf.unary_union.convex_hull
default_buffer_m = 5000   # 20 km buffer; adjust smaller (e.g., 5000) or larger as you wish
area_poly = hull.buffer(default_buffer_m)

# 4) Desired initial cell size (meters) and safety limits
initial_cell_size = 250   # 500 m cells is a good compromise; change to 250 for higher resolution if small domain
max_cells_allowed = 200000  # safety cap to avoid creating millions of cells
cell_size = initial_cell_size

# 5) Compute grid dims and automatically increase cell_size if grid too big
minx, miny, maxx, maxy = area_poly.bounds
nx = int(np.ceil((maxx - minx) / cell_size))
ny = int(np.ceil((maxy - miny) / cell_size))
n_cells = nx * ny
print(f"Initial grid estimate: nx={nx}, ny={ny}, cells≈{n_cells:,}, using cell_size={cell_size} m")

# If too many cells, increase cell_size until below threshold (exponential step)
while n_cells > max_cells_allowed:
    cell_size *= 2
    nx = int(np.ceil((maxx - minx) / cell_size))
    ny = int(np.ceil((maxy - miny) / cell_size))
    n_cells = nx * ny
    print(f"Grid too large — increasing cell_size → {cell_size} m (cells≈{n_cells:,})")

print("Final grid parameters -> cell_size:", cell_size, "meters ; estimated cells:", n_cells)

# 6) Build the grid (only inside buffered hull)
xs = np.arange(minx, maxx + cell_size, cell_size)
ys = np.arange(miny, maxy + cell_size, cell_size)
polygons = []
gids = []
gid = 0
for x in xs[:-1]:
    x2 = x + cell_size
    for y in ys[:-1]:
        y2 = y + cell_size
        poly = Polygon([(x,y),(x2,y),(x2,y2),(x,y2)])
        # add only if intersects the buffered hull (keeps grid small)
        if poly.intersects(area_poly):
            polygons.append(poly)
            gids.append(gid)
            gid += 1

grid = gpd.GeoDataFrame({'grid_id': gids, 'geometry': polygons}, crs='epsg:3857')
print("Built grid cells (kept inside buffered convex hull):", len(grid))

# 7) Save and quick sanity info
grid.to_file(os.path.join(WORK_DIR, "stats19_grid_safe.geojson"), driver="GeoJSON")
print("Saved grid to:", os.path.join(WORK_DIR, "stats19_grid_safe.geojson"))

# Show bounding info and a quick sample
print("Grid bounds (m):", grid.total_bounds)
print("Accident centroid (EPSG:3857):", gdf.geometry.unary_union.centroid)
print(grid.head())
# =======================================================


Rows before filter: 100927 after filter: 100927


C:\Users\Croma\AppData\Local\Temp\ipykernel_23204\3454666037.py:21: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  hull = gdf.unary_union.convex_hull


Initial grid estimate: nx=4113, ny=8224, cells≈33,825,312, using cell_size=250 m
Grid too large — increasing cell_size → 500 m (cells≈8,458,384)
Grid too large — increasing cell_size → 1000 m (cells≈2,115,624)
Grid too large — increasing cell_size → 2000 m (cells≈529,420)
Grid too large — increasing cell_size → 4000 m (cells≈132,612)
Final grid parameters -> cell_size: 4000 meters ; estimated cells: 132612
Built grid cells (kept inside buffered convex hull): 94735
Saved grid to: /content/stats19_mumbai_demo\stats19_grid_safe.geojson
Grid bounds (m): [-827503.73285704 6426091.10456322  204496.26714296 8482091.10456322]
Accident centroid (EPSG:3857): POINT (-136872.53595579087 6873490.58809501)
   grid_id                                           geometry
0        0  POLYGON ((-827503.733 7770091.105, -823503.733...
1        1  POLYGON ((-827503.733 7774091.105, -823503.733...
2        2  POLYGON ((-827503.733 7778091.105, -823503.733...
3        3  POLYGON ((-827503.733 7782091.105, -82

C:\Users\Croma\AppData\Local\Temp\ipykernel_23204\3454666037.py:73: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  print("Accident centroid (EPSG:3857):", gdf.geometry.unary_union.centroid)


In [11]:
gdf = gdf.to_crs(epsg=3857)
join = gpd.sjoin(gdf[['geometry']], grid[['grid_id','geometry']], how='inner', predicate='within')
counts = join.groupby('grid_id').size().reset_index(name='acc_count')
grid = grid.merge(counts, on='grid_id', how='left').fillna({'acc_count':0})
print("Non-zero grid cells:", (grid['acc_count']>0).sum())


Non-zero grid cells: 15010


In [13]:
# Road network via osmnx for the bbox of interest
# Use the polygon we actually constructed: area_poly
bbox_poly = gpd.GeoSeries([area_poly], crs='epsg:3857').to_crs(epsg=4326).iloc[0]

# Download road network inside polygon
G = ox.graph_from_polygon(bbox_poly, network_type='drive')
edges = ox.graph_to_gdfs(G, nodes=False, edges=True).to_crs(epsg=3857)
edges['length_m'] = edges.geometry.length
print("Downloaded roads:", len(edges))
edges = ox.graph_to_gdfs(G, nodes=False, edges=True).to_crs(epsg=3857)
edges['length_m'] = edges.geometry.length

# road length per grid cell
edges_grid = gpd.sjoin(edges[['length_m','geometry']], grid[['grid_id','geometry']], how='inner', predicate='intersects')
road_feats = edges_grid.groupby('grid_id').agg({'length_m':'sum'}).reset_index().rename(columns={'length_m':'total_road_len'})
grid = grid.merge(road_feats, on='grid_id', how='left').fillna({'total_road_len':0})

# intersections (nodes)
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False).to_crs(epsg=3857)
nodes_grid = gpd.sjoin(nodes[['geometry']], grid[['grid_id','geometry']], how='inner', predicate='within')
node_counts = nodes_grid.groupby('grid_id').size().reset_index(name='n_intersections')
grid = grid.merge(node_counts, on='grid_id', how='left').fillna({'n_intersections':0})

# POI counts (targeted amenities) - fetch hospitals/schools/etc using ox.features_from_polygon in tiles for the bbox
# (Use the same tiled approach if bbox is large; for UK tile size can be more generous)
try:
    amen = ox.features_from_polygon(bbox_poly, tags={"amenity": True})
    amen = amen.to_crs(epsg=3857)
    amen = amen[~amen.geometry.is_empty]
    pgrid = gpd.sjoin(amen[['geometry']], grid[['grid_id','geometry']], how='inner', predicate='within')
    poi_counts = pgrid.groupby('grid_id').size().reset_index(name='poi_count')
    grid = grid.merge(poi_counts, on='grid_id', how='left').fillna({'poi_count':0})
except Exception as e:
    print("POI fetch failed:", e)
    grid['poi_count'] = 0

print("Added features. Sample:")
print(grid[['grid_id','acc_count','total_road_len','n_intersections','poi_count']].head())


C:\Users\Croma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\osmnx\_overpass.py:271: UserWarning: This area is 201 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


KeyboardInterrupt: 

In [14]:
print("Active grid cells:", (grid['acc_count']>0).sum())
active_union = grid[grid['acc_count']>0].geometry.buffer(1000).unary_union
print("Active union bounds (EPSG:3857):", active_union.bounds)


Active grid cells: 15010


C:\Users\Croma\AppData\Local\Temp\ipykernel_23204\2324029805.py:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  active_union = grid[grid['acc_count']>0].geometry.buffer(1000).unary_union


Active union bounds (EPSG:3857): (-824503.7328570402, 6429091.104563221, 197496.26714295975, 8479091.104563221)


In [15]:
# ===== FAST & SAFE ROAD NETWORK EXTRACTION =====
import geopandas as gpd
from shapely.ops import unary_union
import osmnx as ox
import os

print("Active accident cells:", (grid['acc_count'] > 0).sum())

# 1) Select only cells with accidents
active_cells = grid[grid['acc_count'] > 0].copy()

# 2) Buffer each grid cell by 800–1000 m to include nearby roads
buffer_m = 1000
active_buffered = active_cells.geometry.buffer(buffer_m)

# 3) Merge into single polygon (much smaller than full UK bbox)
fetch_poly = unary_union(active_buffered)
print("Reduced polygon bounds:", fetch_poly.bounds)

# 4) Convert to WGS84 for Overpass
fetch_poly_wgs = gpd.GeoSeries([fetch_poly], crs="epsg:3857").to_crs(4326).iloc[0]

# 5) Configure OSMnx for long queries
ox.settings.timeout = 180
ox.settings.use_cache = True
ox.settings.log_console = True

print("Downloading small-area road network...")
G = ox.graph_from_polygon(fetch_poly_wgs, network_type='drive')

# Convert to GeoDataFrame
edges = ox.graph_to_gdfs(G, nodes=False, edges=True).to_crs(epsg=3857)
edges["length_m"] = edges.geometry.length

print("Downloaded roads:", len(edges))
print("Saving...")
edges.to_file(os.path.join(WORK_DIR, "roads_small.geojson"), driver="GeoJSON")

print("FAST road extraction completed.")


Active accident cells: 15010
Reduced polygon bounds: (-824503.7328570402, 6429091.104563221, 197496.26714295975, 8479091.104563221)


C:\Users\Croma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\osmnx\_overpass.py:271: UserWarning: This area is 200 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


MemoryError: 